In [ ]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper


llm = ChatOpenAI(
    temperature=0.1,
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query : str = Field(
        description="The Query you will search for. Example quesry: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name : str ="StockMarketSymbolSearchTool"
    description : str = """
    Use this toll to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema : Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Examples: AAPL, TSLA",)

class CompanFinancialOverviewTool(BaseTool):
    name: str="CompanyOverview"
    description : str = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Overview&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name : str = "CompanyIncomeStatement"
    description : str = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get (f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool):
    name : str = "CompanyStockPerformance"
    description : str = """
    Use this to get the Company stock persformance.
    You should enter a stock symbol
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Time_SERIES_MONTHLY_Adjusted&symbol={symbol}&apikey={alpha_vantage_api_key}")

        return r.json()["Monthly Adjusted Time Series"]



agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanFinancialOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ]
)

prompt = "Give me information on Beyond meat's stock, and help me analyze if it's a potential good investment considering its financials, income statement, stock performance."
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Beyond Meat'}`


Get a real-time Beyond Meat, Inc. (BYND) stock price quote with breaking news, financials, statistics, charts and more. Get a real-time Beyond Meat, Inc. (BYND) stock price quote with breaking news, financials, statistics, charts and more. ... Beyond Meat stock has been one of the market's worst performers, losing over half its value due to ... What is Beyond Meat, Inc.'s stock symbol? Beyond Meat, Inc.'s stock symbol is BYND and currently trades under NASDAQ. It's current price per share is approximately $4.08. Beyond Meat Unveils Expanded Line of Beyond Steak®, One of the Fastest Growing and Top Selling Plant-Based Meat Products on the Market (1) GlobeNewswire Feb 6, 2025 2:00pm In the past three months, Beyond Meat insiders have sold more of their company's stock than they have bought. Specifically, they have bought $0.00 in company stock and sol

{'input': "Give me information on Beyond meat's stock, and help me analyze if it's a potential good investment considering its financials, income statement, stock performance.",
 'output': "### Beyond Meat Inc. (BYND) Overview:\n- **Description:** Beyond Meat, Inc. is a food company that manufactures, markets, and sells plant-based meat products in the United States and internationally.\n- **Sector:** Manufacturing\n- **Industry:** Food and Kindred Products\n- **Market Capitalization:** $242.03 million\n- **Revenue (TTM):** $326.45 million\n- **Gross Profit (TTM):** $41.70 million\n- **EPS (TTM):** -$4.19\n- **Profit Margin:** -0.491\n- **Operating Margin (TTM):** -0.493\n- **Return on Assets (TTM):** -0.134\n- **Return on Equity (TTM):** -11.67\n- **Website:** [Beyond Meat Official Site](https://www.beyondmeat.com)\n\n### Financial Analysis:\n- **PE Ratio:** Not available\n- **PEG Ratio:** 40.75\n- **Price to Sales Ratio (TTM):** 0.741\n- **Price to Book Ratio:** 59.32\n- **Forward PE